In [1]:
import pandas as pd

In [2]:
# https://www.kaggle.com/datasets/emreokcular/turkish-song-lyrics
df = pd.read_csv('turkish_song_lyrics.csv')

In [5]:
df.head()

,singer,song,album,lyrics
0,Sagopa Kajmer,3 Sefil Şair (feat Derin Darbe),Romantizma,"""Fakir bir şairim ama,Yüreğim zengin a canımFa..."
1,Zeki Müren,Hangi Rüzgar Attı Seni,Gözlerin Doğuyor Gecelerime,"""Hangi rüzgar attı seni\nÖzlemişim görmeyeli\n..."
2,Serdar Ortaç,Namert Dünyasın,NaN,"""Sözüm dinle dünya bir müddet\nO kadar da işle..."
3,Sagopa Kajmer,Bebeğim Öldü,Romantizma,"""Ve bebekler de ölür""\nHayatımın gerçek öyküle..."
4,Tarkan,Bounce,Yine Sensiz,"""if you"" \nlet me see you bounce (x2) \nif you..."


In [ ]:
# CREATE TABLE songtable (song_id serial primary key, singer VARCHAR(70), song VARCHAR(70), album VARCHAR(70), lyrics TEXT);

In [14]:
# required library imports
import pandas as pd
import psycopg2

import yaml
import warnings

warnings.filterwarnings('ignore')

# read connection.yaml
with open('connection.yaml', 'r') as file:
    infos = yaml.safe_load(file)

# connect database
conn = psycopg2.connect(user= infos['user'],
                        password= infos['password'],
                        host= infos['host'],
                        database= infos['database'])

In [15]:
def read_songtable(query):
    df = pd.read_sql(query, conn)
    return df

In [16]:
read_songtable('SELECT * FROM songtable')

,song_id,singer,song,album,lyrics


In [17]:
insert_query = """INSERT INTO songtable (singer, song, album, lyrics) VALUES (%s, %s, %s, %s)"""

In [19]:
def insert_songtable_many(df):
    """Fonksiyon input olarak pandas dataframe alir."""
    
    conn = psycopg2.connect(user= infos['user'],
                            password= infos['password'],
                            host= infos['host'],
                            database= infos['database'])

    cursor = conn.cursor()

    for i in range(len(df)):
        cursor.execute(insert_query, tuple(df.iloc[i, :]))
        if i % 5000 == 0:
            conn.commit()
    conn.commit()

    cursor.close()
    print(f'{len(df)} rows were inserted.')

In [20]:
insert_songtable_many(df)

4975 rows were inserted.


In [21]:
read_songtable('SELECT * FROM SONGTABLE')

,song_id,singer,song,album,lyrics
0,1,Sagopa Kajmer,3 Sefil Şair (feat Derin Darbe),Romantizma,"""Fakir bir şairim ama,Yüreğim zengin a canımFa..."
1,2,Zeki Müren,Hangi Rüzgar Attı Seni,Gözlerin Doğuyor Gecelerime,"""Hangi rüzgar attı seni\nÖzlemişim görmeyeli\n..."
2,3,Serdar Ortaç,Namert Dünyasın,NaN,"""Sözüm dinle dünya bir müddet\nO kadar da işle..."
3,4,Sagopa Kajmer,Bebeğim Öldü,Romantizma,"""Ve bebekler de ölür""\nHayatımın gerçek öyküle..."
4,5,Tarkan,Bounce,Yine Sensiz,"""if you"" \nlet me see you bounce (x2) \nif you..."
...,...,...,...,...,...
4970,4971,Sagopa Kajmer,Kurşun Asker,Pessimist EP 4,ŞİKAYETNAME\nYAPILACAK İŞ EKİLMEYE HAZIR BAHÇE...
4971,4972,Duman,Bu Aşk Beni Yorar,NaN,"şarkılar, o şarkılar\növünmek zor, ölüm çok zo..."
4972,4973,Candan Erçetin,Bir Dalda İki Kiraz,Aman Doktor,"Σάλα σάλα, μες στη σάλα τα μιλήσαμε \..."
4973,4974,Sagopa Kajmer,Yirmi Dört,NaN,”Acımasızca geçip giden zamandan geriye kalan ...
